In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from my_setting import my_pd_setting, my_np_setting, my_sns_setting, data_path
from my_stat import missing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

my_np_setting()
my_sns_setting()
my_pd_setting()


numpy的版本为1.20.3
seaborn的版本为: 0.11.2
pandas的版本为: 1.3.4


In [2]:
filepath = r'C:\Users\86199\Documents\Datas\Mechine_learing\book1'
data_path(filepath)
df = pd.read_csv('wdbc.csv', header=None)
df.head(2)

当前数据路径为: C:\Users\86199\Documents\Datas\Mechine_learing\book1


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.8,1001.0,0.12,0.28,0.30,0.15,0.24,0.08,1.09,0.91,8.59,153.40,6.40e-03,0.05,0.05,0.02,0.03,6.19e-03,25.38,17.33,184.6,2019.0,0.16,0.67,0.71,0.27,0.46,0.12
1,842517,M,20.57,17.77,132.9,1326.0,0.08,0.08,0.09,0.07,0.18,0.06,0.54,0.73,3.40,74.08,5.22e-03,0.01,0.02,0.01,0.01,3.53e-03,24.99,23.41,158.8,1956.0,0.12,0.19,0.24,0.19,0.28,0.09


In [3]:
# X = df.loc[:,2:] 不加df.values将数据转换为np.array会报错
# y = df.loc[:,1]
X = df.loc[:, 2:].values
y1 = df.loc[:, 1]
y = df.loc[:, 1].values
la = LabelEncoder()
y_l = la.fit_transform(y)
print(len(y1))
print(f'''y包含的类标签为:  {la.classes_}''')
print(f'''列表['M', 'B']的编码情况为:   {la.transform(['M','B'])}''')


569
y包含的类标签为:  ['B' 'M']
列表['M', 'B']的编码情况为:   [1 0]


In [4]:
y1.shape

(569,)

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y_l,test_size=0.2,random_state=1, stratify=y)


In [6]:
pipe_lr = make_pipeline(
    StandardScaler(),
    PCA(n_components=2),
    LogisticRegression(random_state=1, solver='lbfgs')
)
pipe_lr.fit(X_train,y_train)
y_pred = pipe_lr.predict(X_test)
print(f'精度为: {pipe_lr.score(X_test, y_test)}')

精度为: 0.956140350877193


In [7]:
kfold = StratifiedKFold(n_splits=2, random_state=1, shuffle=True).split(X_train, y_train)
print(kfold)
print(list(kfold))
print(type(kfold))
# for k, (tr,te) in enumerate(kfold): # 通过K折获取训练集测试集的索引
#     print('=='*50)
#     print(tr)
#     print(te)

<generator object _BaseKFold.split at 0x0000018002080E40>
[(array([  2,   3,   4,   5,   7,   8,   9,  10,  12,  14,  15,  16,  19,
        20,  25,  27,  28,  30,  33,  35,  36,  41,  48,  49,  50,  51,
        52,  53,  56,  59,  60,  65,  70,  72,  76,  77,  80,  84,  87,
        88,  89,  93,  94,  96,  98, 100, 101, 102, 104, 108, 109, 110,
       113, 115, 116, 118, 119, 121, 123, 125, 127, 129, 132, 133, 135,
       138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 151, 153, 155,
       161, 166, 167, 169, 170, 174, 175, 176, 177, 179, 180, 181, 182,
       183, 185, 186, 187, 188, 191, 199, 200, 204, 206, 208, 213, 214,
       216, 217, 218, 219, 220, 223, 225, 229, 230, 231, 233, 235, 237,
       238, 239, 240, 241, 242, 245, 246, 250, 252, 257, 258, 259, 260,
       261, 262, 263, 264, 265, 270, 271, 274, 276, 278, 280, 283, 284,
       286, 287, 288, 290, 291, 293, 294, 297, 299, 300, 303, 307, 308,
       309, 310, 311, 313, 314, 316, 320, 321, 322, 324, 325, 327, 328,
    

In [8]:
# https://stackoverflow.com/questions/59895070/keyerror-none-of-int64index-dtype-int64-length-9313-are-in-the-columns
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

Fold:  1, Class dist.: [256 153], Acc: 0.935
Fold:  2, Class dist.: [256 153], Acc: 0.935
Fold:  3, Class dist.: [256 153], Acc: 0.957
Fold:  4, Class dist.: [256 153], Acc: 0.957
Fold:  5, Class dist.: [256 153], Acc: 0.935
Fold:  6, Class dist.: [257 153], Acc: 0.956
Fold:  7, Class dist.: [257 153], Acc: 0.978
Fold:  8, Class dist.: [257 153], Acc: 0.933
Fold:  9, Class dist.: [257 153], Acc: 0.956
Fold: 10, Class dist.: [257 153], Acc: 0.956

CV accuracy: 0.950 +/- 0.014


In [48]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/'
                 'machine-learning-databases'
                 '/breast-cancer-wisconsin/wdbc.data', header=None)

# if the Breast Cancer dataset is temporarily unavailable from the
# UCI machine learning repository, un-comment the following line
# of code to load the dataset from a local path:

# df = pd.read_csv('wdbc.data', header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.12,0.28,0.30,0.15,0.24,0.08,1.09,0.91,8.59,153.40,6.40e-03,0.05,0.05,0.02,0.03,6.19e-03,25.38,17.33,184.60,2019.0,0.16,0.67,0.71,0.27,0.46,0.12
1,842517,M,20.57,17.77,132.90,1326.0,0.08,0.08,0.09,0.07,0.18,0.06,0.54,0.73,3.40,74.08,5.22e-03,0.01,0.02,0.01,0.01,3.53e-03,24.99,23.41,158.80,1956.0,0.12,0.19,0.24,0.19,0.28,0.09
2,84300903,M,19.69,21.25,130.00,1203.0,0.11,0.16,0.20,0.13,0.21,0.06,0.75,0.79,4.58,94.03,6.15e-03,0.04,0.04,0.02,0.02,4.57e-03,23.57,25.53,152.50,1709.0,0.14,0.42,0.45,0.24,0.36,0.09
3,84348301,M,11.42,20.38,77.58,386.1,0.14,0.28,0.24,0.11,0.26,0.10,0.50,1.16,3.44,27.23,9.11e-03,0.07,0.06,0.02,0.06,9.21e-03,14.91,26.50,98.87,567.7,0.21,0.87,0.69,0.26,0.66,0.17
4,84358402,M,20.29,14.34,135.10,1297.0,0.10,0.13,0.20,0.10,0.18,0.06,0.76,0.78,5.44,94.44,1.15e-02,0.02,0.06,0.02,0.02,5.11e-03,22.54,16.67,152.20,1575.0,0.14,0.20,0.40,0.16,0.24,0.08


In [54]:
df.describe()

,0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
count,5.69e+02,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,5.69e+02,569.00,569.00,569.00,569.00,5.69e+02,5.69e+02,569.00,5.69e+02,5.69e+02,5.69e+02,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00
mean,3.04e+07,14.13,19.29,91.97,654.89,0.10,0.10,0.09,0.05,0.18,6.28e-02,0.41,1.22,2.87,40.34,7.04e-03,2.55e-02,0.03,1.18e-02,2.05e-02,3.79e-03,16.27,25.68,107.26,880.58,0.13,0.25,0.27,0.11,0.29,0.08
std,1.25e+08,3.52,4.30,24.30,351.91,0.01,0.05,0.08,0.04,0.03,7.06e-03,0.28,0.55,2.02,45.49,3.00e-03,1.79e-02,0.03,6.17e-03,8.27e-03,2.65e-03,4.83,6.15,33.60,569.36,0.02,0.16,0.21,0.07,0.06,0.02
min,8.67e+03,6.98,9.71,43.79,143.50,0.05,0.02,0.00,0.00,0.11,5.00e-02,0.11,0.36,0.76,6.80,1.71e-03,2.25e-03,0.00,0.00e+00,7.88e-03,8.95e-04,7.93,12.02,50.41,185.20,0.07,0.03,0.00,0.00,0.16,0.06
25%,8.69e+05,11.70,16.17,75.17,420.30,0.09,0.06,0.03,0.02,0.16,5.77e-02,0.23,0.83,1.61,17.85,5.17e-03,1.31e-02,0.02,7.64e-03,1.52e-02,2.25e-03,13.01,21.08,84.11,515.30,0.12,0.15,0.11,0.06,0.25,0.07
50%,9.06e+05,13.37,18.84,86.24,551.10,0.10,0.09,0.06,0.03,0.18,6.15e-02,0.32,1.11,2.29,24.53,6.38e-03,2.04e-02,0.03,1.09e-02,1.87e-02,3.19e-03,14.97,25.41,97.66,686.50,0.13,0.21,0.23,0.10,0.28,0.08
75%,8.81e+06,15.78,21.80,104.10,782.70,0.11,0.13,0.13,0.07,0.20,6.61e-02,0.48,1.47,3.36,45.19,8.15e-03,3.24e-02,0.04,1.47e-02,2.35e-02,4.56e-03,18.79,29.72,125.40,1084.00,0.15,0.34,0.38,0.16,0.32,0.09
max,9.11e+08,28.11,39.28,188.50,2501.00,0.16,0.35,0.43,0.20,0.30,9.74e-02,2.87,4.88,21.98,542.20,3.11e-02,1.35e-01,0.40,5.28e-02,7.90e-02,2.98e-02,36.04,49.54,251.20,4254.00,0.22,1.06,1.25,0.29,0.66,0.21


In [49]:
from sklearn.preprocessing import LabelEncoder

X = df.loc[:, 2:].values
y = df.loc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

array(['B', 'M'], dtype=object)

In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, 
                     test_size=0.20,
                     stratify=y,
                     random_state=1)

In [51]:

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(random_state=1, solver='lbfgs'))

pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
print('Test Accuracy: %.3f' % pipe_lr.score(X_test, y_test))

Test Accuracy: 0.956


In [52]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
    

kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)

scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

Fold:  1, Class dist.: [256 153], Acc: 0.935
Fold:  2, Class dist.: [256 153], Acc: 0.935
Fold:  3, Class dist.: [256 153], Acc: 0.957
Fold:  4, Class dist.: [256 153], Acc: 0.957
Fold:  5, Class dist.: [256 153], Acc: 0.935
Fold:  6, Class dist.: [257 153], Acc: 0.956
Fold:  7, Class dist.: [257 153], Acc: 0.978
Fold:  8, Class dist.: [257 153], Acc: 0.933
Fold:  9, Class dist.: [257 153], Acc: 0.956
Fold: 10, Class dist.: [257 153], Acc: 0.956

CV accuracy: 0.950 +/- 0.014
